This is a random forest for the same dataset for checking accuracy.
so here is my problem i have text of news articles i want to 
generate a bias_rating = left right center and 
i want type of bias = conservative, liberal, center, socialist etc 
i have 2 strategies 

1 -> sequential that is i will first train a bias classifier on just text to predict a left right center label then i will train another random forest on text and bias to provide type of bias 
2-> use a multilabel mutlioutput strategy to predict both simultaneously 

i have the test and training data in csv with rows 
text
bias_rating 
type_of_biasness

In [21]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

Load Data

In [35]:
data = pd.read_csv('dataset.csv')

data["bias_label"] = data["bias_rating"].map({
    "Left": 0,
    "Center": 1,
    "Right": 2
})

data["subtype_label"] = data["type_of_biasness"].map({
    "Conservative": 0,
    "Capitalist": 1,
    "Nationalist": 2,
    "Center" : 3,
    "Liberal" : 4,
    "Secular" : 5,
    "Socialist" : 6
})



Data Split

In [36]:
X = data["text"]
y = data[["bias_label", "subtype_label"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


TF-IDF Vectorization along with Stop Words Removal and Word Lemmatization

In [37]:
clean = []
lemma = WordNetLemmatizer()
swords = stopwords.words("english")

for line in X_train:
    line = re.sub(r'http\S+', '', line)
    line = re.sub("[^a-zA-Z0-9 ]"," ",line)
    line = line.lower()
    line = nltk.word_tokenize(line)
    line = [lemma.lemmatize(word) for word in line if word not in swords]
    line = " ".join(line)
    clean.append(line)

In [38]:
clean_test = []
for line in X_test:
    line = re.sub(r'http\S+', '', line)
    line = re.sub("[^a-zA-Z0-9 ]"," ",line)
    line = line.lower()
    line = nltk.word_tokenize(line)
    line = [lemma.lemmatize(word) for word in line if word not in swords]
    line = " ".join(line)
    clean_test.append(line)

In [ ]:
vectorizer = TfidfVectorizer(max_features=7000)
X_train_tfidf = vectorizer.fit_transform(clean)
X_test_tfidf = vectorizer.transform(clean_test)

Random Forest Classification for Multilabel output

In [ ]:
rf = RandomForestClassifier(n_estimators=250, random_state=42)
multi_rf = MultiOutputClassifier(rf)
multi_rf.fit(X_train_tfidf, y_train)


,estimator,RandomForestC...ndom_state=42)
,n_jobs,None
,n_estimators,500
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0


Predict

In [48]:
prediction = multi_rf.predict(X_test_tfidf)

bias_pred = prediction[:, 0]
subtype_pred = prediction[:, 1]

Evaluation

In [49]:
print("Bias Classification Report:")
print(classification_report(y_test["bias_label"], bias_pred))
print("Bias Accuracy:", accuracy_score(y_test["bias_label"], bias_pred))
print("Bias Confusion Matrix:")
print(confusion_matrix(y_test["bias_label"], bias_pred))
print("------------------------------------------------------------")
print("Subtype Classification Report:")
print(classification_report(y_test["subtype_label"], subtype_pred))
print("Subtype Accuracy:", accuracy_score(y_test["subtype_label"], subtype_pred))
print("Subtype Confusion Matrix:")
print(confusion_matrix(y_test["subtype_label"], subtype_pred))

Bias Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.77      0.72       641
           1       0.38      0.08      0.14       181
           2       0.65      0.71      0.68       578

    accuracy                           0.66      1400
   macro avg       0.57      0.52      0.51      1400
weighted avg       0.63      0.66      0.63      1400

Bias Accuracy: 0.6592857142857143
Bias Confusion Matrix:
[[496   4 141]
 [ 87  15  79]
 [146  20 412]]
------------------------------------------------------------
Subtype Classification Report:
              precision    recall  f1-score   support

           0       0.38      0.27      0.32       204
           1       0.83      0.96      0.89       203
           2       0.57      0.83      0.68       171
           3       0.35      0.33      0.34       181
           4       0.42      0.30      0.35       215
           5       0.96      0.95      0.95       208
           6     

In [ ]:
joblib.dump(multi_rf, "multioutput_rf_model.joblib")
joblib.dump(vectorizer, "multioutput_vectorizer.joblib")

['tfidf_vectorizer.joblib']